In [1]:
import sys
sys.path.append("/storage/vbutoi/projects/")
sys.path.append("/storage/vbutoi/libraries/")
from ionpy.analysis import ResultsLoader
import pathlib
import seaborn as sns
import pandas as pd
sns.set_context('talk')
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
path_list = [
    # "/storage/vbutoi/scratch/ESE/calibration/09_11_24_ISLES_3D_LTS_AndrewsLevelsBIG",
    # "/storage/vbutoi/scratch/ESE/calibration/09_12_24_ISLES_3D_LTS_LRandCapSweep",
    "/storage/vbutoi/scratch/ESE/calibration/09_16_24_ISLES_3D_LTS_wAug",
    # "/storage/vbutoi/scratch/ESE/calibration/09_17_24_ISLES_3D_LTS_CE_wAug",
    # "/storage/vbutoi/scratch/ESE/calibration/09_17_24_ISLES_3D_LTS_SoftDice_wAug",
]

df_list = []
for path in path_list:
    try:
        dfc = rs.load_configs(
            path,
            properties=False,
        )
        df = rs.load_metrics(dfc)
        df_list.append(df)
    except Exception as e:
        print(e)
df = pd.concat(df_list)

In [3]:
# Only keep the epochs every 10 to reduce the number of points
df = df[df['epoch'] % 10 == 0]

In [4]:
# for ikey in df.keys():
#     print(ikey)

In [5]:
# g = sns.relplot(
#     data=df,
#     x='epoch',
#     y='loss',
#     hue='loss_func',
#     style='seed',
#     # hue='filters',
#     # style='lr',
#     col='phase',
#     kind='line',
#     height=12,
# )

# # Set column spacing
# g.fig.subplots_adjust(wspace=0.05)
# # g.set(ylim=(0.0, 0.25))

In [6]:
# g = sns.relplot(
#     data=df,
#     x='epoch',
#     y='area_estimation_error',
#     hue='loss_func',
#     style='seed',
#     # hue='filters',
#     # style='lr',
#     col='phase',
#     kind='line',
#     height=12,
#     # palette=custom_palette
# )

# # Set column spacing
# g.fig.subplots_adjust(wspace=0.05)
# # g.set(ylim=(0.0, 0.25))

In [7]:
def aee_squared(area_estimation_error):
    return area_estimation_error**2

def abs_area_est_error(area_estimation_error):
    return abs(area_estimation_error)

def loss_rooted(loss):
    return loss**0.5

df.augment(aee_squared)
df.augment(abs_area_est_error)
df.augment(loss_rooted)

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='loss_rooted',
    hue='loss_func',
    style='seed',
    col='phase',
    kind='line',
    height=12,
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# Set the y lim to be between 0 and 1000
g.set(ylim=(0.0, 1000))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='abs_area_est_error',
    hue='loss_func',
    style='seed',
    col='phase',
    kind='line',
    height=12,
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
g.set(ylim=(0.0, 1000))